In [ ]:
import numpy as np
import scipy.stats as stats
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


path = "/content/"
#TimeSeriesForestClassifier	RISE	ROCKET	KNeighborsTimeSeriesClassifier	InceptionTime	XGBClassifier	SVC	DecisionTreeClassifier	KNeighborsClassifier	RandomForestClassifier
columns = ["TSFC","RISE","ROCKET","KNTC","Inception","XGBC","SVC","DTC","KNNC","RFC"]
# columns = ["KNN","BOSS","HIVECOTE1","HIVECOTE2","RISE","ROCKET","STC","TSFC","Inception"]
columnsDict = {1:"TSFC",2:"RISE",3:"ROCKET",4:"KNNTC",5:"InceptionTime"
               ,6:"XGBC",7:"SVC",8:"DTC",9:"KNNC", 10:'RFC'}

df_antes = pd.read_csv(path+'AllAccuraciesBloodNoOversample.csv',sep=';', names=columns, skiprows=1)
df_depois = pd.read_csv(path+'AllAccuraciesBloodOversample.csv',sep=';', names=columns, skiprows=1)

# 📌 Garantindo que as colunas estão na mesma ordem
df_depois = df_depois[df_antes.columns]

# 📌 Extraindo os classificadores (nomes das colunas)
classificadores = df_antes.columns.tolist()

# 📌 Convertendo para matriz NumPy
grupo1 = df_antes.to_numpy()
grupo2 = df_depois.to_numpy()

# Criando DataFrame para armazenar os resultados
resultados = []

# Loop para cada classificador
for i, nome in enumerate(classificadores):
    # Teste de normalidade (Shapiro-Wilk)
    p_grupo1 = stats.shapiro(grupo1[:, i]).pvalue
    p_grupo2 = stats.shapiro(grupo2[:, i]).pvalue

    # Escolha do teste estatístico
    if p_grupo1 > 0.05 and p_grupo2 > 0.05:
        stat, p_valor = stats.ttest_rel(grupo1[:, i], grupo2[:, i])
        teste_usado = "paired t-test"
    else:
        stat, p_valor = stats.wilcoxon(grupo1[:, i], grupo2[:, i])
        teste_usado = "Wilcoxon"

    # Adiciona os resultados à lista
    resultados.append([nome, p_grupo1, p_grupo2, teste_usado, p_valor])

# Converte os resultados para DataFrame
df_resultados = pd.DataFrame(resultados, columns=["Classificador", "Shapiro Grupo 1", "Shapiro Grupo 2", "Teste", "p-valor"])
df_resultados.to_csv("testeNormalidade.csv")
print("Valores salvos")
# 📌 Função para detalhar os resultados
def interpretar_resultados(df, alpha=0.05):
    significativos = df[df["p-valor"] <= alpha]
    nao_significativos = df[df["p-valor"] > alpha]

    print("📊 **Classificadores com diferença estatisticamente significativa após o oversampling:**")
    if not significativos.empty:
        print(significativos[["Classificador", "p-valor", "Teste"]])
    else:
        print("Nenhum classificador apresentou diferença estatisticamente significativa. 😐")

    print("\n📌 **Classificadores sem diferença significativa:**")
    if not nao_significativos.empty:
        print(nao_significativos[["Classificador", "p-valor", "Teste"]])
    else:
        print("Todos os classificadores apresentaram diferença significativa!")

# 📊 Plotando o gráfico dos p-valores
def plotar_resultados(df, alpha=0.05):
    plt.figure(figsize=(12, 6))
    sns.barplot(x="Classificador", y="p-valor", hue="Teste", data=df, palette="coolwarm", dodge=False)

    # Linha de referência para p-valor = 0.05
    plt.axhline(y=alpha, color="red", linestyle="--", label="Significance (p=0.05)")

    plt.xticks(rotation=45, ha="right")
    plt.ylim(0, 1)
    plt.xlabel("Classifiers")
    plt.ylabel("p-values")
    plt.title("Comparison of p-values ​​for Oversampling in Classifiers")
    plt.legend()
    plt.show()

# 📊 Função para criar um heatmap de p-valores
def plotar_heatmap(df, alpha=0.05):
    plt.figure(figsize=(10, 4))
    df_pvalores = df.set_index("Classificador")[["p-valor"]].T  # Transforma para melhor visualização

    sns.heatmap(df_pvalores, cmap="coolwarm", annot=True, fmt=".3f", linewidths=0.5, center=alpha)

    plt.title("Heatmap de p-valores - Oversampling nos Classificadores")
    plt.xlabel("Classificadores")
    plt.ylabel("p-valor")
    plt.show()
# Chamando as funções
interpretar_resultados(df_resultados)
plotar_resultados(df_resultados)
plotar_heatmap(df_resultados)
